# 2/27 Notebook - Customer Support Chatbot

Hello and welcome to this week's notebook! Last week, we learned how to create a custom data set, professionally clean data, and train a chat bot using a bag-of-words model. This week, we'll be making predictions from our model and creating a user interface for our chatbot

The solutions to last week's notebook are incorporated into this notebook, so feel free to look through the methods to refresh yourself on how everything works. There will be a heading indicating where the new code begins

There's nothing you need to change in the first part, but if you created a custom `intents.json`, make sure you replace that file here

**Note: This notebook does requires the additional installation of `Keras` and `Tensorflow`. If you don't want to install these libraries or are having trouble, try the other notebook!**

Below are the methods you need to complete for the notebook:
1. `predict_tag()`
2. `get_response()`
3. `chat()`

**Note: You can skip reading many of the following cells, but make sure you run them so your model is trained**

We'll start by importing our libraries as always. Make sure you run the cell with `pip install nltk`, which will let you download the `nltk` library we'll be using

In [1]:
!pip install nltk

In [2]:
# import our nltk libraries
import nltk
from nltk.stem import WordNetLemmatizer
# install specific downloads
nltk.download('punkt', quiet = True)
nltk.download('wordnet', quiet = True)

True

In [3]:
# other useful libraries (numpy == 🐐)
import numpy as np
import random
import json

## Part 1: Modify your intents

The great part about this chat bot is that it is fully customizable! Edit `intents.json` to your liking to create your own bot. Make sure that for each `intent`, you fill out the fields `tag`, `patterns`, and `responses`

You can look at my file, `taco-bell-intents.json`, for reference

Once you're done, you can continue to run the cells below!

**Note: if you're having JSON formatting issues in the next cell, use [this link](https://jsonlint.com) to validate your JSON**

In [9]:
data_file = open("intents.json").read()
intents = json.loads(data_file)
# when you print, you should see your JSON
print(intents)

{'intents': [{'tag': 'google', 'patterns': ['google', 'search', 'internet'], 'responses': ['Redirecting to Google...']}, {'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Namaste', 'yo'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide

## Part 2: Parsing the JSON

We'll practice a common first step in any NLP project, data cleaning

First, complete the function `process_words()` which will clean up our words according to the following steps:
1. Get the tokens using `nltk.word_tokenize()`
2. Set `cleaned_word` equal to the `lemmatized` and `lowercased` word

**Note: Make sure you run the cell immediately below this first; it stores values needed in `process_words()`**

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Set <code>tokens = nltk.word_tokenize(pattern)</code></li>
    <li><code>lemmatizer.lemmatize(...)</code> will lemmatize a word</li>
    <li>The paremeter of <code>lemmatizer.lemmatize(...)</code> should be <code>word.lower()</code></li>  
</ul>
</p>

In [10]:
# declare needed variables for process_words()
ignore_punctuation = ["?", "!", ".", ","]
lemmatizer = WordNetLemmatizer()

In [11]:
def process_words(pattern):
    # return variable
    words = []
    # get the tokens using nltk
    tokens = nltk.word_tokenize(pattern)
    for word in tokens:
        # check if the word should be ignored
        if word not in ignore_punctuation and word.isalnum():
            # clean the word and add it to the list
            cleaned_word = lemmatizer.lemmatize(word.lower())
            words.append(cleaned_word)
    # return the list
    return words

In [12]:
# run this cell to test your code
if (process_words("How was your day today?") == ['how', 'wa', 'your', 'day', 'today']):
    print("Nice work, sport!")
else:
    print("Try again, buddy!")

Nice work, sport!


Now that we have `process_words()` to clean our words, we can parse the data from our JSON

Complete the method `parse_intents()` which does the following:
1. Set the value of `tag` from our `intent`
2. Set `tokenized_words` using the helper method in `process_words()`
3. Append a tuple of `tokenized_words` and `tag` to `tag_tokens`

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>Values of a JSON can be extracted using arrays</li>
    <li>Let <code>tag = intent["tag"]</code></li>
    <li>Let <code>tokenized_words = process_words(pattern)</code></li>
    <li>For the third step, the tuple can be appended with <code>tag_tokens.append((tokenized_words, tag))</code></li>
</ul>
</p>

In [13]:
def parse_intents(intents):
    # declare our needed variables
    tags = []
    all_words = []
    tag_tokens = []
    response_dict = dict()
    
    # iterate through each intent
    for intent in intents["intents"]:
        
        # add the noanswer tag to the dictionary (edge case)
        if (intent["tag"] == "noanswer"):
            response_dict["noanswer"] = intent["responses"]
        
        # if the intent has no patterns, we can skip
        if (len(intent["patterns"]) == 0):
            continue
        
        # add the tag to the list of tag
        tag = intent["tag"]
        tags.append(tag)
        
        # update the dictionary
        response_dict[tag] = intent["responses"]
        
        # iterate through each pattern
        for pattern in intent["patterns"]:
            # create our tokenized words
            tokenized_words = process_words(pattern)
            # add all the tokenized words to our words
            all_words.extend(tokenized_words)
            # adds a tuple -> (list of tokens, tag) -> to the list
            tag_tokens.append((tokenized_words, tag))
    # return our values in a tuple
    return (np.array(tags), np.array(all_words), np.array(tag_tokens), response_dict)

We can do this cool trick below to remove all duplicates from our arrays (and sort them)

In [14]:
# call our function
tags, all_words, tag_tokens, tag_responses = parse_intents(intents)
# sort and remove duplicates
tags = np.array(sorted(list(set(tags))))
all_words = np.array(sorted(list(set(all_words))))

C:\Users\azaan\AppData\Local\Temp\ipykernel_151928\1739760297.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return (np.array(tags), np.array(all_words), np.array(tag_tokens), response_dict)


Run the cell below and take a quick look to make sure that everything makes sense. It's hard for me to test your code without knowing what's in your JSON, but in general:

- `tags` should contain a list of all your tags in the JSON, excluding `noanswer`
- `all_words` should be a list of all the words in your JSON's patterns. There should be no duplicates or patterns that aren't words
- Each entry of `tag_token_mappings` should have two values in a list. The first should be a list of patterns, and the second should be the tag of that pattern

In [15]:
print("Tags: {0}".format(tags))
print("------")
print("All Words: {0}".format(all_words))
print("------")
print("Tag-Token Mappings: {0}".format(tag_tokens))

Tags: ['Identity' 'activity' 'age' 'appreciate' 'contact' 'covid19' 'cricket'
 'datetime' 'exclaim' 'goodbye' 'google' 'greeting' 'greetreply' 'haha'
 'inspire' 'insult' 'jokes' 'karan' 'news' 'nicetty' 'no' 'options'
 'programmer' 'riddle' 'song' 'suggest' 'thanks' 'timer' 'weather'
 'whatsup']
------
All Words: ['10' '19' 'a' 'age' 'am' 'anyone' 'are' 'ask' 'awesome' 'bad' 'bbye' 'be'
 'best' 'bye' 'can' 'contact' 'could' 'covid' 'creator' 'cricket'
 'current' 'date' 'day' 'designed' 'developer' 'do' 'doing' 'dumb' 'fine'
 'for' 'funny' 'get' 'good' 'goodbye' 'google' 'great' 'haha' 'he' 'hello'
 'help' 'helpful' 'helping' 'hey' 'hi' 'hola' 'hot' 'how' 'i' 'idiot'
 'india' 'inspiration' 'inspires' 'internet' 'is' 'it' 'joke' 'karan'
 'know' 'later' 'latest' 'laugh' 'lmao' 'lol' 'lost' 'made' 'make' 'malik'
 'match' 'me' 'motivates' 'namaste' 'news' 'next' 'nice' 'no' 'nope'
 'offered' 'ok' 'old' 'programmed' 'programmer' 'provide' 'question'
 'riddle' 'rofl' 'score' 'search' 'see' 's

## Part 3: Creating a Training Set

We know from previous lessons that the computer can't train a model without numeric values. To solve this, we'll use the `bag of words` technique we discussed in the Google Sheets



Complete the helper method `build_bag()` which iterates through each `word` in `all_words`, and appends 1 to `bag` if the word is in `all_words`, and 0 otherwise

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>The easiest way to do this is by using a simple <code>if else</code> statement</li>
    <li>Recall that <code>A in B</code> will return <code>true</code> if the element A is in the iterable object B, and <code>false</code> otherwise</li>
    <li>If you're feeling really fancy, you can just write <code>bag.append(1 * (word in tokens))</code></li>
</ul>
</p>

In [16]:
def build_bag(all_words, tokens):
    # reset our current bag
    bag = []
    for word in all_words:
        # add 0/1 if the word is in our token
        in_token = (word in tokens)
        bag.append(1 * in_token)
    return bag

In [17]:
# run this cell to test your code
test_all_words = ["edgar", "allen", "poe", "said", "the", "raven", "was", "nevermore"]
test_tokens = ["quote", "the", "raven", "nevermore"]
if (build_bag(test_all_words, test_tokens) == [0, 0, 0, 0, 1, 1, 0, 1]):
    print("You crushed it!")
else:
    print("Ruh roh raggy")

You crushed it!


Complete the method `build_training_set()` below, which performs the following steps:
1. Grabs the value of `tokens`, the first (index 0) element of `tag_token`
2. Grabs the value of `tag`, the second (index 1) element of `tag_token`
3. Sets `current_bag` using the helper method `build_bag()`

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<p>
<ul>
    <li>You can get the values of <code>tokens</code> and <code>tag</code> with <code>tag_token[X]</code>, where <code>X</code> is 0 or 1, appropriately</li>
    <li>Let <code>current_bag = build_bag(all_words, tokens)</code></li>
</ul>
</p>

In [18]:
def build_training_set(tags, all_words, tag_tokens):
    # define our variables to return
    train_x = []
    train_y = []
        
    # iterate through each tag-token mapping
    for tag_token in tag_tokens:
        
        # grab our needed values
        tokens = tag_token[0]
        tag = tag_token[1]
        
        # reset our current bag
        current_bag = build_bag(all_words, tokens)
            
        # update our training inputs
        train_x.append(current_bag)
        
        # set our outputs equal to 1 in the location
        train_y.append(1 * (tags == tag))
    
    # return our values
    return (np.array(train_x), np.array(train_y))

In [19]:
train_x, train_y = build_training_set(tags, all_words, tag_tokens)

Print your `train_x` and `train_y` values in the following cell. It's hard for me to tell if you did everything correctly since you could be using a custom data set. If you have any questions about the program, feel free to message me on discord!

- `train_x` should be dimension `(m, n)` where `m` = # of total patterns and `n` = # words in `all_words`
- `train_y` should be dimension `(m, n)` where `m` = # of total patterns and `n` = # tags in `tags`

In [20]:
print(train_x.shape)
print(train_y.shape)
print("Training Inputs: {0}".format(train_x))
print("-----")
print("Training Outputs: {0}".format(train_y))

(113, 127)
(113, 30)
Training Inputs: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]
-----
Training Outputs: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]


Before we continue with training, you may notice that our data is very similarly grouped, specifically the training outputs. As you may have thought, this can cause some unwanted bias in our model. To fix this, we'll `shuffle` our training set by using `np.random.permutation()` and some clever array indexing:

In [21]:
# shuffled indexes
shuffled_indexes = np.random.permutation(train_x.shape[0])
# set new values for train_x and train_y
train_x = train_x[shuffled_indexes]
train_y = train_y[shuffled_indexes]

## Part 4: Training Our Model Using Keras/Tensorflow (no coding)

We have our cleaned, numeric inputs and outputs (`train_x` and `train_y`), so now what? 

It's time to train our model!

**Note: In this version of the notebook, we'll be using `Tensorflow` and `Keras`. I have some instructions below on how to set this up. If you're still having trouble, switch over to the other notebook as there's no installation required**

1. Open `Anaconda Prompt`
2. `conda install pip`
3. `pip install --upgrade tensorflow`
4. `pip install Keras`
5. `conda create -n mnist tensorflow keras`
6. `conda activate mnist`
7. `conda install jupyter`
8. `conda list` - verify that you see jupyter, numpy, keras, and tensorflow
9. run `jupyter notebook` and open this file again

Hopefully, your installation worked without too much trouble. If you can run the next cell without any errors, you should be good to go! As always, if you have any questions you can message me on Discord

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

If the installation worked properly, you'll see a message that reads `Using Tensorflow backend.`

Now, we'll use `Keras` to create a `Sequential` model. This library makes it very easy for us to create convolutional neural networks

Our model will use the following architecture:

<img src = "./bag_of_words.PNG" style="width:75%;"></img>

In [23]:
# declare our model
model = Sequential()
# add our layers
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

You may see a lot of unfamiliar terms in this model, so I'll do my best to define what the above cell does:
- `model.add(Dense(...))` adds a layer of neurons to our neural network. The number is the size of our network, but can be overriden by `input_shape`
- `Dropout(0.5)` adds `regularization` to our model, something we haven't talked about yet. Basically, `regularization` decreases the likelihood of the model overfitting our data. Overfitting occurs when our model can predict our training set very well, but does poorly with new data
- `activation = 'relu'` changes the activation function. Before, we were using `sigmoid`, but `relu` is another popular function. You can read more about it [here](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning)
- In many neural networks, the final activation function is `softmax`, which essentially normalizes our data. You can read more about it [here](https://en.wikipedia.org/wiki/Softmax_function)

Next, we'll create an optimizer using `stochastic gradient descent`. The algorithm we were using in earlier weeks was `batch gradient descent`. The main difference between the two optimizers is that `batch gradient descent` takes the derivative of the entire data set at once, while `stochastic gradient descent` takes the partial derivative of each entry in the data set one at a time

The parameters `lr`, `decay`, `momentum`, and `nesterov` adjusts how fast our model will train. With these parameters set, our model will train more slowly over time

We set our `loss` function to [categorical_crossentropy](https://gombru.github.io/2018/05/23/cross_entropy_loss/), our `optimizer` to `stochastic gradient descent`, and tell the model to print out the `accuracy` during each iteration

In [24]:
sgd = SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

c:\Users\azaan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


`Keras` makes it very easy to train our model. We can use `model.fit()` to accomplish this. Some notes about the parameters:
- `epochs` is equivalent to our number of iterations
- `batch_size` tells our model how often to compute the partial derivatives
- setting `verbose` to 1 just displays a progress bar

Run the cell below to visualize the training of our model!

In [25]:
hist = model.fit(train_x, train_y, epochs = 500, batch_size = 5, verbose = 1)

Epoch 1/500
23/23 [==============================] - 0s 2ms/step - loss: 3.4484 - accuracy: 0.0088
Epoch 2/500
23/23 [==============================] - 0s 3ms/step - loss: 3.3623 - accuracy: 0.0796
Epoch 3/500
23/23 [==============================] - 0s 1ms/step - loss: 3.2886 - accuracy: 0.1062
Epoch 4/500
23/23 [==============================] - 0s 1ms/step - loss: 3.2503 - accuracy: 0.0885
Epoch 5/500
23/23 [==============================] - 0s 1ms/step - loss: 3.2340 - accuracy: 0.1239
Epoch 6/500
23/23 [==============================] - 0s 1ms/step - loss: 3.1578 - accuracy: 0.1150
Epoch 7/500
23/23 [==============================] - 0s 1ms/step - loss: 3.1110 - accuracy: 0.1150
Epoch 8/500
23/23 [==============================] - 0s 2ms/step - loss: 3.0321 - accuracy: 0.1150
Epoch 9/500
23/23 [==============================] - 0s 1ms/step - loss: 2.9403 - accuracy: 0.2124
Epoch 10/500
23/23 [==============================] - 0s 2ms/step - loss: 2.9518 - accuracy: 0.1681
Epoch 11/

# Part B: Finishing the Chatbot

Now that we have everything we need, we can finish the chatbot!

As a reminder, these are the methods you need to complete:
1. `predict_tag()`
2. `get_response()`
3. `chat()`

Again, this is quite a short notebook, so feel free to spend some time looking over and finishing the first part if you need to!

## Part 1: Predicting the Tag

Last week we created a neural network that took our training data as an input, and matched it to a tag output. Now, we need to predict tags using custom user inputs

Complete the method `predict_tag()`, which does the following
1. Sets `process_input` to the cleaned and tokenized `user_input`
2. Sets `bag_input` to the bag representation of `process_input`
3. Calculates `pred_tag_values` by using our `model` and the `predict()` function
4. Finds the values for `max_value_tag` and `probability`, the maximum index and value, respectively, of `pred_tag_values`
5. Gets the value of `pred_tag` using `max_value_tag` and the list of `tags`

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 1</b></font>
</summary>
<p>
<ul>
    <li>Use the <code>process_words()</code> helper function with <code>user_input</code> as the parameter</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 2</b></font>
</summary>
<p>
<ul>
    <li>Use the <code>build_bag()</code> helper function with <code>all_words</code> and <code>process_input</code> as the parameters</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 3</b></font>
</summary>
<p>
<ul>
    <li><code>model.predict()</code> will get the values you need</li>
    <li>The argument for <code>model.predict()</code> should be <code>bag_input</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 4</b></font>
</summary>
<p>
<ul>
    <li><code>np.argmax()</code> will return the index of the maximum value of a numpy array. <a href = https://numpy.org/doc/stable/reference/generated/numpy.argmax.html>Documentation</a></li>
    <li><code>np.max()</code> will return the maximum value of a numpy array. <a href = https://numpy.org/doc/stable/reference/generated/numpy.amax.html>Documentation</a></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 5</b></font>
</summary>
<p>
<ul>
    <li><code>pred_tag</code> will be the value of the numpy array <code>tags</code> at index <code>max_value_tag</code></li>
</ul>
</p>

In [26]:
def predict_tag(user_input, model):
    # [your code here] - tokenize/clean inputs
    process_input = process_words(user_input)
    
    # [your code here] - build the bag
    bag_input = build_bag(all_words, process_input)
    bag_input = np.array([bag_input]) # note: convert to a numpy array
    
    # [your code here] - get our predicted values
    pred_tag_values = model.predict(bag_input)
    pred_tag_values = pred_tag_values[0] # note: flatten the 2-d array
    
    # [your code here] - get the index and value of the largest probability value
    max_value_tag = np.argmax(pred_tag_values)
    probability = np.max(pred_tag_values)
    
    # [your code here] - predict the tag and return
    pred_tag = tags[max_value_tag]
    return (pred_tag, probability)

In [28]:
# try your own inputs here and make sure that the tag makes sense!
# look at the probability for the bot's confidence level
custom_input = "How are you today?"
predict_tag(custom_input, model)

1/1 [==============================] - 0s 20ms/step


('greeting', 0.50441056)

## Part 2: Getting a Response

Now that we have a way to predict our tags, we need to get a user input from this tag 

Complete the method `get_response()`, which does the following
1. Sets `pred_tag` and `probability` using our helper method `predict_tag()`
2. Gets the correct responses from the `tag_responses` dictionary. If the `probability` is not high enough, it should equal `tag_responses["noanswer"]`
3. Sets the boolean value of `should_exit_bot`, which should be `true` if the predicted tag is `"goodbye"` or `"thanks"`
4. Get a random `response` from our `responses`

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 1</b></font>
</summary>
<p>
<ul>
    <li>Use the <code>predict_tag()</code> helper function with <code>user_input</code> and <code>thetas</code> as the parameters</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 2</b></font>
</summary>
<p>
<ul>
    <li>Write your code in the format <code>responses = [_____] if [_____] else [_____]</code></li>
    <li>The first blank should be the value from the dictionary, <code>tag_responses[pred_tag]</code></li>
    <li>The second blank should be the condition when <code>probability > error_margin</code></li>
    <li>The final blank should be the invalid response tag, <code>tag_responses["noanswer"]</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 3</b></font>
</summary>
<p>
<ul>
    <li>The value should be <code>true</code> with the goodbye tag, so set <code>should_exit_bot</code> equal to <code>pred_tag == "goodbye" or pred_tag == "thanks"</code></li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 4</b></font>
</summary>
<p>
<ul>
    <li><code>np.random.choice()</code> will return a random element from an array. Read more about it <a href = https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html>here</a></li>
</ul>
</p>

In [29]:
def get_response(user_input, model, error_margin):
    # [your code here] - get the predicted tag and probability
    pred_tag, probability = predict_tag(user_input, model)
    
    # [your code here] - get a list of different responses
    responses = tag_responses[pred_tag] if probability > error_margin else tag_responses["noanswer"]
    
    # [your code here] - check if we should exit the bot
    should_exit_bot = pred_tag == 'goodbye' or pred_tag == "thanks"
    
    # [your code here] - get the response
    response = np.random.choice(responses)
    
    # return the variables
    return (response, should_exit_bot)

In [31]:
# try your own inputs here and check the responses! (remember the second output should only be true for goodbye messages)
custom_input = "How are you today?"
get_response(custom_input, model, 0.25)

1/1 [==============================] - 0s 23ms/step


('Good to see you again', False)

## Part 3: Creating the Chat User Interface

Everything is set up! We just need to put it all together

Complete the method `chat()`, which puts it all together:
1. Sets `user_input` using Python's `input()` function. I recommend putting in `human_prefix` as the parameter. Read more about it [here](https://www.w3schools.com/python/ref_func_input.asp)
2. Finds `response` and `should_exit` using our helper method `get_response`
3. Prints the `response` preceded by the `robot_prefix`
4. Sets `continue_chat` appropriately

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints for Step 1</b></font>
</summary>
<p>
<ul>
    <li>If you set <code>user_input</code> equal to <code>input()</code>, it will automatically store the user's input</li>
    <li>Set the parameter of <code>input()</code> to be <code>human_prefix</code> to improve your UI</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 2</b></font>
</summary>
<p>
<ul>
    <li>Call <code>get_response()</code> with <code>user_input</code>, <code>thetas</code> and <code>0.25</code> as the parameters. The last value can be anything, depending on how accurate you want your bot to be</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 3</b></font>
</summary>
<p>
<ul>
    <li>Use string concatenation to combine the strings</li>
    <li>Print <code>robot_prefix + response</code> to the console</li>
</ul>
</p>

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint for Step 4</b></font>
</summary>
<p>
<ul>
    <li>We should continue when we shouldn't exit, so set <code>continue_chat = not should_exit</code></li>
</ul>
</p>

In [32]:
def chat():
    # initialize variables
    continue_chat = True
    robot_prefix = "Bot: "
    human_prefix = "You: "
    
    # give an introduction
    print(robot_prefix + "Hi! I am a bot offering support for Taco Bell. Ask me what I can do! To exit, say goodbye")
    print("")
    
    # continue while the user doesn't say goodbye
    while (continue_chat):
        # [your code here] - get the user input from the console
        user_input = input(human_prefix)
        
        # [your code here] - get the response and exit condition from the helper function
        response, should_exit = get_response(user_input, model, 0.25)
        
        # [your code here] - print the bot's response 
        print(robot_prefix +  response)
        print("")
        
        # [your code here] - set the exit condition
        continue_chat = not should_exit

And that's everything you need! If you made it this far, congratulations, you just made your first chat bot! Run the method below to test it out

If you want to add more patterns/responses to the bot, you can modify `intents.json` to your liking

In [33]:
chat()

Bot: Hi! I am a bot offering support for Taco Bell. Ask me what I can do! To exit, say goodbye

1/1 [==============================] - 0s 17ms/step
Bot: Good to know!

1/1 [==============================] - 0s 18ms/step
Bot: o what if I don't know what 'Armageddon' means? It's not the end of the world.

1/1 [==============================] - 0s 22ms/step
Bot: All good..What about you?

1/1 [==============================] - 0s 18ms/step
Bot: What has a thumb and four fingers but is not actually alive?.....Your Gloves!

1/1 [==============================] - 0s 17ms/step
Bot: Yeah!



KeyboardInterrupt: Interrupted by user